## youtube8Mのデータセット準備

とりあえずローカルでDatasetAPIで学習を開始できるようにするのが目的

### ローカルへのデータダウンロード

youtube8mのデータは以下のコマンドで取って来れます。
下のセルを実行すると全データの1/100のトレーニングデータを取得します。

環境変数からオプションを渡すような作りになってるみたい。
このpartition=2の部分がよくわからないけど、とりあえずこのままでw

In [ ]:
BUCKET="asl-mixi-project-bucket"

In [ ]:
%%bash 
CURRENT_DIR=$(PWD)
mkdir ../data/video/
cd ../data/video/
curl data.yt8m.org/download.py > download.py

shard=1,100 partition=2/video/train mirror=us python download.py
cd $CURRENT_DIR

### storageへのコピー

必要があれば実行。ローカルで実行する分には必要ありません。

In [ ]:
import os
os.environ['BUCKET'] = BUCKET

!gsutil cp ../data/video/*.tfrecord gs://$BUCKET/data/youtube-8m/

### データセットの作成用のコード

In [ ]:
import tensorflow as tf

print(tf.__version__)

In [ ]:
tf.enable_eager_execution()

In [ ]:
def parse_row(row):
  context_features = {
      "id": tf.FixedLenFeature([], tf.string),
      "labels": tf.VarLenFeature(tf.int64),
      "mean_rgb": tf.VarLenFeature(tf.float32),
      "mean_audio": tf.VarLenFeature(tf.float32)
  }
  return tf.parse_single_sequence_example(row, context_features)

def make_dataset(files_pattern, mode, batch_size=128):
    tffiles = tf.io.gfile.glob(files_pattern)
    dataset = tf.data.TFRecordDataset(tffiles).map(parse_row)
    if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.shuffle(batch_size*10).repeat().batch(batch_size)
    else:
        dataset = dataset.repeat(1).batch(batch_size)
    return dataset

In [ ]:
ds = make_dataset("gs://asl-mixi-project-bucket/data/youtube_8m/*.tfrecord", tf.estimator.ModeKeys.TRAIN, 1)

In [ ]:
for row in ds.take(3):
    print(row)